1、导入pandas库

In [2]:
import pandas as pd

2、导入数据集airlines

In [3]:
airlines_csv = 'https://raw.githubusercontent.com/zhoujinhai/pandasData/master/airlines.csv'

3、将数据集存入一个名为airlines的数据框内

In [4]:
airlines = pd.read_csv(airlines_csv, sep=',')

4、查看后面5行行内容

In [5]:
print(airlines.tail())

                       airline  avail_seat_km_per_week  incidents_85_99  \
51       United / Continental*              7139291291               19   
52  US Airways / America West*              2455687887               16   
53            Vietnam Airlines               625084918                7   
54             Virgin Atlantic              1005248585                1   
55             Xiamen Airlines               430462962                9   

    fatal_accidents_85_99  fatalities_85_99  incidents_00_14  \
51                      8               319               14   
52                      7               224               11   
53                      3               171                1   
54                      0                 0                0   
55                      1                82                2   

    fatal_accidents_00_14  fatalities_00_14  
51                      2               109  
52                      2                23  
53                      0 

5、观察数据集的信息

In [6]:
print(airlines.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   airline                 56 non-null     object
 1   avail_seat_km_per_week  56 non-null     int64 
 2   incidents_85_99         56 non-null     int64 
 3   fatal_accidents_85_99   56 non-null     int64 
 4   fatalities_85_99        56 non-null     int64 
 5   incidents_00_14         56 non-null     int64 
 6   fatal_accidents_00_14   56 non-null     int64 
 7   fatalities_00_14        56 non-null     int64 
dtypes: int64(7), object(1)
memory usage: 3.6+ KB
None


6、打印数据集大小

In [7]:
print(airlines.shape)

(56, 8)


7、打印出全部的列名称

In [8]:
print(airlines.columns)

Index(['airline', 'avail_seat_km_per_week', 'incidents_85_99',
       'fatal_accidents_85_99', 'fatalities_85_99', 'incidents_00_14',
       'fatal_accidents_00_14', 'fatalities_00_14'],
      dtype='object')


In [ ]:
"""
====================================
真实数据集的异常值检测简介
====================================

此示例介绍了对真实数据集进行鲁棒协方差估计的必须性。(鲁棒的意思是足够稳定，绝不过拟合。)
它对于异常检测(离群值检测)和更好地理解数据结构都是有用的。

我们从波士顿住房数据集中选择了两组两个变量的数据子集，
以说明可以使用几种离群值检测工具进行哪些分析。
出于可视化的目的，我们使用的是2维示例，但是应该指出的是，在高维度上事情并非那么简单。

在下面的两个示例来看，主要结论是经验协方差估计(作为一种非稳健的估计)
受观测的异构结构的影响很大。尽管鲁棒的协方差估计能够集中于数据分布的主要模式，
但它假定数据应该是高斯分布的，从而产生了对数据结构的某种有偏估计，不过在一定程度上还算准确。
One-Class SVM不假设数据分布的任何参数形式，因此可以更好地对数据的复杂形状进行建模。

第一个例子
-------------
第一个示例说明了当另外一个簇存在时，鲁棒的协方差估计如何帮助集中在相关簇上。
在这里，许多观察结果被混淆为一个，让经验协方差估计效果变差。
当然，某些筛选工具能指出存在两个聚类(支持向量机，高斯混合模型，单变量离群值检测……)。
但是，如果这是一个高维度的例子，那么所有这些都不容易被应用。

第二个例子
-------------
第二个示例显示了最小协方差鲁棒估计器专注于数据分布的主要模式的能力：
尽管由于香蕉形分布而难以估算协方差，但位置似乎已得到很好的估计。
无论如何，我们可以消除一些较远的离群点。 One-Class SVM能够捕获真实的数据结构，
但是困难在于如何调整其核带宽参数，以便在数据散布矩阵的形状和过滤合数据的风险之间取得良好的折衷。
"""

# Author: Virgile Fritsch <virgile.fritsch@inria.fr>
# License: BSD 3 clause

# 载入此项目所需要的库
import numpy as np
# 椭圆模型拟合(EllipticEnvelope),它能拟合出数据的稳健协方差估计，从而为中心数据点拟合出一个椭圆，忽略不和该中心模式相关的点
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM  # 单类支持向量机
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn.datasets import load_boston  # 从 sklearn 数据集中导入波士顿房价数据

# 获取数据 取数据集里的两个变量，这两个变量的观测组成两类。
X1 = load_boston()['data'][:, [8, 10]]  # 将数据分成两个集群
X2 = load_boston()['data'][:, [5, 12]]  # 香蕉形

# 定义要使用的分类器对象 classifiers, 它是一个字典型，
# 由 Empirical Covariance, Robust Covariance, 单类支持向量机 OCSVM 三个分类器组成。
# 最小协方差决定（Minimum Covariance Determinant）这个想法是找出一个给定比例（h）的 “好” 观察值，它们不是离群值，
# 且可以计算其经验协方差矩阵。 然后将该经验协方差矩阵重新缩放以补偿所执行的观察选择（”consistency step(一致性步骤)”）。
# 计算完最小协方差决定估计器后，可以根据其马氏距离（Mahalanobis distance）给出观测值的权重, 得到数据集的协方差矩阵的重新加权估计（”reweighting step(重新加权步骤)”）
classifiers = {
    "Empirical Covariance": EllipticEnvelope(  # EmpiricalCovariance 是基于最大似然协方差估计的算法
        support_fraction=1.,  # 支持原始MCD(最小协方差行列式)估算的点数比例
        contamination=0.261),  # 数据集的污染量，即数据集中异常值的比例。
    # Robust Covariance是基于最小协方差行列式的算法，是鲁棒协方差估计。
    "Robust Covariance (Minimum Covariance Determinant)":
    EllipticEnvelope(contamination=0.261),
        "OCSVM": OneClassSVM(nu=0.261,  # nu训练误差分数的上限，支持向量分数的下限
                             gamma=0.05)}  # gamma 内核系数
colors = ['m', 'g', 'b']
legend1 = {}
legend2 = {}

# 使用定义的三个分类器，确定异常点检测边界
# scikit-learn项目提供了一套可用于新奇点或离群点检测的机器学习工具。
# 该策略是以无监督的方式学习数据中的对象来实现的: estimator.fit(X_train)
# 然后可以使用 decision_function 方法将新观测值归为内围点或离群点: estimator.decision_function(X_test)
# 内围点被标记为1，而离群点被标记为-1。 预测方法在估计器计算出的原始评分函数上使用一个阈值。
# 这个阈值可以由参数contamination控制。
# numpy.meshgrid()从坐标向量中返回坐标矩阵
# numpy.linspace 函数用于创建一个一维数组，数组是一个等差数列构成的
# numpy.linspace(start, stop, num) 目的是在start-stop之间生成num个数
# numpy.c_是按行连接两个矩阵，就是把两矩阵左右相加，要求行数相等
xx1, yy1 = np.meshgrid(np.linspace(-8, 28, 500), np.linspace(3, 40, 500))
xx2, yy2 = np.meshgrid(np.linspace(3, 10, 500), np.linspace(-5, 45, 500))
for i, (clf_name, clf) in enumerate(classifiers.items()):
    plt.figure(1)
    clf.fit(X1)  # 模型拟合
    Z1 = clf.decision_function(  # decision_function代表的是参数实例到各个类所代表的超平面的距离
        np.c_[xx1.ravel(), yy1.ravel()])
    Z1 = Z1.reshape(xx1.shape)  # 将Z1的维度指定为xx1的维度
    legend1[clf_name] = plt.contour(  # 绘制等高线
        xx1, yy1, Z1,  # 使用 xx1 和 yy1 绘制 Z1 的等高线图，指定 Z1 中各值的 x 和 y 坐标
        levels=[0],  # levels表示等高线的密集程度，如果是0，则图像被一分为二
        linewidths=2,  # 线的宽度
        colors=colors[i])  # 标签的颜色
    plt.figure(2)

    clf.fit(X2)
    Z2 = clf.decision_function(np.c_[xx2.ravel(), yy2.ravel()])
    Z2 = Z2.reshape(xx2.shape)
    legend2[clf_name] = plt.contour(  # 绘制等高线
        xx2, yy2, Z2,  # 使用 xx2 和 yy2 绘制 Z2 的等高线图，指定 Z2 中各值的 x 和 y 坐标
        levels=[0], linewidths=2, colors=colors[i])

legend1_values_list = list(legend1.values())
legend1_keys_list = list(legend1.keys())

# 画出结果图，我们会看到一个明显异常的观测点。
# 使用plt.figure定义一个图像窗口：编号为1；
# 使用plt.title定义标题；
# 使用plt.scatter绘制散点，点的颜色属性(color)为黑色；
# 使用plt.annotate绘制注解；
# 使用plt.xlim, plt.ylim设置坐标轴的取值范围;
# 使用plt.legend绘制图例；
# 使用plt.xlabel, plt.ylabel定义标签
plt.figure(1)  # two clusters
plt.title("Outlier detection on a real data set (boston housing)")  # 标题
plt.scatter(X1[:, 0], X1[:, 1], color='black')  # 绘制散点
bbox_args = dict(boxstyle="round", fc="0.8")
arrow_args = dict(arrowstyle="->")
# 绘制注解
plt.annotate("several confounded points",  # 注释文本内容
             xy=(24, 19),  # 被注释的坐标点(箭头点所在的坐标位置)
             xycoords="data",  # **coords 指定坐标
             textcoords="data",  # data：使用被注释对象的坐标系统(默认)
             xytext=(13, 10),  # 注释文字的坐标位置
             bbox=bbox_args,  # 为注释文本添加边框
             arrowprops=arrow_args)  # 设置指向箭头的参数,参数类型为字典dict
# 设置坐标轴的取值范围
plt.xlim((xx1.min(), xx1.max()))
plt.ylim((yy1.min(), yy1.max()))
# 设置 legend 图例
plt.legend((legend1_values_list[0].collections[0],  # handles参数，用于指定图例上面的图形
            legend1_values_list[1].collections[0],
            legend1_values_list[2].collections[0]),
           (legend1_keys_list[0], legend1_keys_list[1],
            legend1_keys_list[2]),  # labels参数，图例上图形对应的标签
           loc="upper center",  # 图例的位置
           prop=matplotlib.font_manager.FontProperties(size=12))  # 图例的字体属性
# 设置坐标轴的标签label
plt.ylabel("accessibility to radial highways")
plt.xlabel("pupil-teacher ratio by town")

legend2_values_list = list(legend2.values())
legend2_keys_list = list(legend2.keys())

# 新建一个名叫 Figure2的画图窗口
plt.figure(2)  # "banana" shape
plt.title("Outlier detection on a real data set (boston housing)")  # 标题
plt.scatter(X2[:, 0], X2[:, 1], color='black')  # 绘制散点
# 设置坐标轴的取值范围
plt.xlim((xx2.min(), xx2.max()))
plt.ylim((yy2.min(), yy2.max()))
# 设置 legend 图例
plt.legend((legend2_values_list[0].collections[0],  # handles参数，用于指定图例上面的图形
            legend2_values_list[1].collections[0],
            legend2_values_list[2].collections[0]),
           (legend2_keys_list[0], legend2_keys_list[1],
            legend2_keys_list[2]),  # labels参数，图例上图形对应的标签
           loc="upper center",  # loc表示位置
           prop=matplotlib.font_manager.FontProperties(size=12))  # prop字体参数
# 设置坐标轴的标签label
plt.ylabel("% lower status of the population")
plt.xlabel("average number of rooms per dwelling")

# 显示
plt.show()